In [2]:
# General data analysis/plotting
import pandas as pd
import numpy as np
import pickle
import math

# Data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# Neural Net modules
from keras.models import Sequential
from keras.layers import Dense

import matplotlib

df = pd.read_excel("data.xlsx")
df = df[['Diagnosis', 'Tindakan', 'SEX', 'UMUR_TAHUN', 'LAMA_DIRAWAT', 'INACBG']]

df['DiagnosisCAT'] = df['Diagnosis']
df['Diagnosis'] = df['DiagnosisCAT'].astype('category')
df['Diagnosis'] = df['Diagnosis'].cat.reorder_categories(df['DiagnosisCAT'].unique(), ordered=True)
df['Diagnosis'] = df['Diagnosis'].cat.codes

df['TindakanCAT'] = df['Tindakan']
df['Tindakan'] = df['TindakanCAT'].astype('category')
df['Tindakan'] = df['Tindakan'].cat.reorder_categories(df['TindakanCAT'].unique(), ordered=True)
df['Tindakan'] = df['Tindakan'].cat.codes

severity = []
for index, row in df.iterrows():
    if row['INACBG'].split("-")[-1] == "I":
        severity.append(1)
    elif row['INACBG'].split("-")[-1] == "II":
        severity.append(2)
    else:
        severity.append(3)

df['Severity'] = severity

df.dropna(axis=0, inplace=True)

X = df[['Diagnosis', 'Tindakan', 'SEX', 'UMUR_TAHUN', 'Severity']].values
y = df[['LAMA_DIRAWAT']].values

### Sandardization of data ###
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# # Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# # Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

batch_size_list=[16, 32, 64, 128]
epoch_list  =   [50, 100, 200, 500]

SearchResultsData=pd.DataFrame(columns=['parameter', 'mse', 'akurasi'])
batch_size =[]
epoch= []
mse_list = []

# initializing the trials
TrialNumber=0
for batch_size_trial in batch_size_list:
    for epochs_trial in epoch_list:
        model = Sequential()
        # Defining the first layer of the model
        model.add(Dense(units=5, input_shape=(5,), kernel_initializer='normal', activation='relu'))

        # Defining the Second layer of the model
        model.add(Dense(units=60, kernel_initializer='normal', activation='relu'))
        model.add(Dense(units=60, kernel_initializer='normal', activation='relu'))
        model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))


        # The output neuron is a single fully connected node 
        # Since we will be predicting a single number
        model.add(Dense(1, kernel_initializer='normal', activation='linear'))

        # Compiling the model
        model.compile(loss='mean_squared_error', optimizer='adam')

        # Fitting the ANN to the Training set
        model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
        mse = model.evaluate(X_test, y_test, verbose=0)
        batch_size.append(batch_size_trial)
        epoch.append(epochs_trial)
        mse_list.append(mse)

        pred = model.predict(X_test)

        y_pred=TargetVarScalerFit.inverse_transform(pred)
        
        # Scaling the y_test Price data back to original price scale
        y_test_orig=TargetVarScalerFit.inverse_transform(y_test)

        APE=100*(abs(y_test_orig - y_pred)/y_test_orig)
        ape_result = 100-np.mean(APE)
        rounded_up = math.ceil(ape_result)
        
        print("Batch: ", batch_size_trial)
        print("epoch: ", epochs_trial)
        print("mse: ", mse)
        print("akurasi, ", ape_result)
        print("")

        SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[ 'btch ' + str(batch_size_trial) + '-' + ' epoch ' + str(epochs_trial), mse, rounded_up]], columns=['parameter', 'mse', 'akurasi']))

# SearchResultsData.plot(x='parameter', y='mse', figsize=(15,4), kind='line')
# SearchResultsData['batch_size'] = batch_size
# SearchResultsData['epoch'] = epoch
# SearchResultsData['mse'] = mse_list
# print(SearchResultsData.head())
SearchResultsData.to_excel("evaluasi_mse.xlsx")

# model.save("data/model.h5")
# print("ok")

3/3 [==============================] - 0s 4ms/step
Batch:  16
epoch:  50
mse:  0.059133272618055344
akurasi,  92.99701687037533



C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4221227721.py:116: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[ 'btch ' + str(batch_size_trial) + '-' + ' epoch ' + str(epochs_trial), mse, rounded_up]], columns=['parameter', 'mse', 'akurasi']))


3/3 [==============================] - 0s 3ms/step
Batch:  16
epoch:  100
mse:  0.01472601667046547
akurasi,  96.75045807922194



C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4221227721.py:116: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[ 'btch ' + str(batch_size_trial) + '-' + ' epoch ' + str(epochs_trial), mse, rounded_up]], columns=['parameter', 'mse', 'akurasi']))


3/3 [==============================] - 0s 4ms/step
Batch:  16
epoch:  200
mse:  0.01628158427774906
akurasi,  96.80299919517316



...

C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4221227721.py:116: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[ 'btch ' + str(batch_size_trial) + '-' + ' epoch ' + str(epochs_trial), mse, rounded_up]], columns=['parameter', 'mse', 'akurasi']))


3/3 [==============================] - 0s 5ms/step
Batch:  128
epoch:  200
mse:  0.05646054446697235
akurasi,  93.13425343782929



C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4221227721.py:116: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[ 'btch ' + str(batch_size_trial) + '-' + ' epoch ' + str(epochs_trial), mse, rounded_up]], columns=['parameter', 'mse', 'akurasi']))


3/3 [==============================] - 0s 5ms/step
Batch:  128
epoch:  500
mse:  0.0187135748565197
akurasi,  97.65335359022139



C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4221227721.py:116: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[ 'btch ' + str(batch_size_trial) + '-' + ' epoch ' + str(epochs_trial), mse, rounded_up]], columns=['parameter', 'mse', 'akurasi']))


In [3]:
# General data analysis/plotting
import pandas as pd
import numpy as np
import pickle
import math

# Data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from keras.optimizers import Adam


# Neural Net modules
from keras.models import Sequential
from keras.layers import Dense

import matplotlib

df = pd.read_excel("data.xlsx")
df = df[['Diagnosis', 'Tindakan', 'SEX', 'UMUR_TAHUN', 'LAMA_DIRAWAT', 'INACBG']]

df['DiagnosisCAT'] = df['Diagnosis']
df['Diagnosis'] = df['DiagnosisCAT'].astype('category')
df['Diagnosis'] = df['Diagnosis'].cat.reorder_categories(df['DiagnosisCAT'].unique(), ordered=True)
df['Diagnosis'] = df['Diagnosis'].cat.codes

df['TindakanCAT'] = df['Tindakan']
df['Tindakan'] = df['TindakanCAT'].astype('category')
df['Tindakan'] = df['Tindakan'].cat.reorder_categories(df['TindakanCAT'].unique(), ordered=True)
df['Tindakan'] = df['Tindakan'].cat.codes

severity = []
for index, row in df.iterrows():
    if row['INACBG'].split("-")[-1] == "I":
        severity.append(1)
    elif row['INACBG'].split("-")[-1] == "II":
        severity.append(2)
    else:
        severity.append(3)

df['Severity'] = severity

df.dropna(axis=0, inplace=True)

X = df[['Diagnosis', 'Tindakan', 'SEX', 'UMUR_TAHUN', 'Severity']].values
y = df[['LAMA_DIRAWAT']].values

### Sandardization of data ###
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# # Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# # Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

batch_size_list=[16, 32, 64, 128]
epoch_list  =   [50, 100, 200, 500]
learning_rate = [0.001, 0.01, 0.1]

SearchResultsData=pd.DataFrame(columns=['batch_size', 'epochs', 'learning_rate', 'mse', 'akurasi'])
batch_size =[]
epoch= []
mse_list = []

# initializing the trials
TrialNumber=0
for batch_size_trial in batch_size_list:
    for epochs_trial in epoch_list:
        for learning_trial in learning_rate:
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=5, input_shape=(5,), kernel_initializer='normal', activation='relu'))

            # Defining the Second layer of the model
            model.add(Dense(units=60, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=60, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))


            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal', activation='linear'))
            
            optimizer = Adam(learning_rate=learning_trial)

            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer=optimizer)

            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            mse = model.evaluate(X_test, y_test, verbose=0)
            batch_size.append(batch_size_trial)
            epoch.append(epochs_trial)
            mse_list.append(mse)

            pred = model.predict(X_test)

            y_pred=TargetVarScalerFit.inverse_transform(pred)
            
            # Scaling the y_test Price data back to original price scale
            y_test_orig=TargetVarScalerFit.inverse_transform(y_test)

            APE=100*(abs(y_test_orig - y_pred)/y_test_orig)
            ape_result = 100-np.mean(APE)
            rounded_up = math.ceil(ape_result)
            
            print("Batch: ", batch_size_trial)
            print("epoch: ", epochs_trial)
            print("learning rate: ", learning_trial)
            print("mse: ", mse)
            print("akurasi, ", ape_result)
            print("")

            SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[batch_size_trial, epochs_trial, learning_trial, mse, rounded_up]], columns=['batch_size', 'epochs', 'learning_rate', 'mse', 'akurasi']))

# SearchResultsData.plot(x='parameter', y='mse', figsize=(15,4), kind='line')
# SearchResultsData['batch_size'] = batch_size
# SearchResultsData['epoch'] = epoch
# SearchResultsData['mse'] = mse_list
# print(SearchResultsData.head())
SearchResultsData.to_excel("evaluasi_mse_with_learning_rate.xlsx")

# model.save("data/model.h5")
# print("ok")

3/3 [==============================] - 0s 5ms/step
Batch:  16
epoch:  50
learning rate:  0.001
mse:  0.11716139316558838
akurasi,  87.33269303468475



C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4276221582.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[batch_size_trial, epochs_trial, learning_trial, mse, rounded_up]], columns=['batch_size', 'epochs', 'learning_rate', 'mse', 'akurasi']))


3/3 [==============================] - 0s 0s/step
Batch:  16
epoch:  50
learning rate:  0.01
mse:  0.03889009729027748
akurasi,  95.20613592498498



C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4276221582.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[batch_size_trial, epochs_trial, learning_trial, mse, rounded_up]], columns=['batch_size', 'epochs', 'learning_rate', 'mse', 'akurasi']))


3/3 [==============================] - 0s 4ms/step
Batch:  16
epoch:  50
learning rate:  0.1
mse:  0.9208301901817322
akurasi,  63.226355389417584



...

C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4276221582.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[batch_size_trial, epochs_trial, learning_trial, mse, rounded_up]], columns=['batch_size', 'epochs', 'learning_rate', 'mse', 'akurasi']))


3/3 [==============================] - 0s 0s/step
Batch:  128
epoch:  500
learning rate:  0.01
mse:  0.020933235064148903
akurasi,  96.99912152019701



C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4276221582.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[batch_size_trial, epochs_trial, learning_trial, mse, rounded_up]], columns=['batch_size', 'epochs', 'learning_rate', 'mse', 'akurasi']))


3/3 [==============================] - 0s 0s/step
Batch:  128
epoch:  500
learning rate:  0.1
mse:  0.9228876829147339
akurasi,  65.42021478165682



C:\Users\muham\AppData\Local\Temp\ipykernel_8952\4276221582.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[batch_size_trial, epochs_trial, learning_trial, mse, rounded_up]], columns=['batch_size', 'epochs', 'learning_rate', 'mse', 'akurasi']))


In [ ]:
# General data analysis/plotting
import pandas as pd
import numpy as np
import pickle

# Data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# Neural Net modules
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV


import matplotlib

df = pd.read_excel("data.xlsx")
df = df[['Diagnosis', 'Tindakan', 'SEX', 'UMUR_TAHUN', 'LAMA_DIRAWAT', 'KELAS_RAWAT']]

df['DiagnosisCAT'] = df['Diagnosis']
df['Diagnosis'] = df['DiagnosisCAT'].astype('category')
df['Diagnosis'] = df['Diagnosis'].cat.reorder_categories(df['DiagnosisCAT'].unique(), ordered=True)
df['Diagnosis'] = df['Diagnosis'].cat.codes

df['TindakanCAT'] = df['Tindakan']
df['Tindakan'] = df['TindakanCAT'].astype('category')
df['Tindakan'] = df['Tindakan'].cat.reorder_categories(df['TindakanCAT'].unique(), ordered=True)
df['Tindakan'] = df['Tindakan'].cat.codes

df.dropna(axis=0, inplace=True)

X = df[['Diagnosis', 'Tindakan', 'SEX', 'UMUR_TAHUN', 'KELAS_RAWAT']].values
y = df[['LAMA_DIRAWAT']].values

### Sandardization of data ###
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# # Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# # Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

batch_size_list=[100, 150, 200, 250]
epoch_list  =   [50, 100, 200, 500]

# initializing the trials
def createModel():
    model = Sequential()
    # Defining the first layer of the model
    model.add(Dense(units=5, input_shape=(5,), kernel_initializer='normal', activation='relu'))

    # Defining the Second layer of the model
    model.add(Dense(units=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))


    # The output neuron is a single fully connected node 
    # Since we will be predicting a single number
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))

    # Compiling the model
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

model = KerasRegressor(model=createModel, verbose=0)

param_grid = dict(batch_size=batch_size_list, epochs=epoch_list)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
grid_pred = grid_result.predict(X_test)

gg = []
for i in grid_pred:
    gg.append(i)

y_valid = y_test.copy()

gf = pd.DataFrame(data=gg)
# gf = gf.set_index(y_valid.index)
gf.rename(columns={0: "predicted"}, inplace=True)
df321 = pd.DataFrame(data=[[y_valid, gf]])
df321.columns = ["Y_true", "Y_pred"]

df321["Squared Error"] = (df321["Y_true"] - df321["Y_pred"] )**2
print(df321['Squared Error'].head())


# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



df = pd.read_excel("data.xlsx")
df = df[['Diagnosis', 'Tindakan', 'SEX', 'UMUR_TAHUN', 'LAMA_DIRAWAT', 'KELAS_RAWAT']]

df['DiagnosisCAT'] = df['Diagnosis']
df['Diagnosis'] = df['DiagnosisCAT'].astype('category')
df['Diagnosis'] = df['Diagnosis'].cat.reorder_categories(df['DiagnosisCAT'].unique(), ordered=True)
df['Diagnosis'] = df['Diagnosis'].cat.codes

df['TindakanCAT'] = df['Tindakan']
df['Tindakan'] = df['TindakanCAT'].astype('category')
df['Tindakan'] = df['Tindakan'].cat.reorder_categories(df['TindakanCAT'].unique(), ordered=True)
df['Tindakan'] = df['Tindakan'].cat.codes

df.dropna(axis=0, inplace=True)

X = df[['Diagnosis', 'Tindakan', 'SEX', 'UMUR_TAHUN', 'KELAS_RAWAT']].values
y = df[['LAMA_DIRAWAT']].values

### Sandardization of data ###
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# # Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

batch_size_list=[100, 150, 200, 250]
epoch_list  =   [50, 100, 200, 500]

SearchResultsData=pd.DataFrame(columns=['parameter', 'mse', 'akurasi'])

# Function to create the model
def create_model():
    model = Sequential()
    # Defining the first layer of the model
    model.add(Dense(units=5, input_shape=(5,), kernel_initializer='normal', activation='relu'))

    # Defining the Second layer of the model
    model.add(Dense(units=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))


    # The output neuron is a single fully connected node 
    # Since we will be predicting a single number
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))

    # Compiling the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


model = KerasRegressor(model=create_model, verbose=0)

# Define the hyperparameters grid
param_grid = {'batch_size': batch_size_list,
              'epochs': epoch_list}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Get the results of grid search
results = grid_result.cv_results_

# Print MSE for each parameter combination
for mean_score, params in zip(results["mean_test_score"], results["params"]):
    model = KerasRegressor(model=create_model, verbose=0, **params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Scaling the predicted Price data back to original price scale
    y_pred_origin=TargetVarScalerFit.inverse_transform(y_pred)
    
    # Scaling the y_test Price data back to original price scale
    y_test_origin=TargetVarScalerFit.inverse_transform(y_test)
    
    mse = mean_squared_error(y_pred= y_pred, y_true= y_test)

    APE=100*(abs(y_test_origin - y_pred_origin)/y_test_origin)
    ape_result = 100-np.mean(APE)
    
    # SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[ 'batch ' + str(params['batch_size']) + '-' + ' epoch ' + str(params['epochs']), mse, ape_result]], columns=['parameter', 'mse', 'akurasi']))
    
    print("Parameters: ", params)
    print("Mean Squared Error: ", mse)
    print("akurasi: ", ape_result)
    print()
